In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import os
from common import *
import pandas as pd
#from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import Lasso, Ridge, LinearRegression

os.chdir("../")
print(f"Root directory: {os.getcwd()}")

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
    "font.size": 10,
})

%matplotlib inline 

Root directory: c:\Users\andre\Dropbox\FYS-STK4155_projects\FYS-STK4155 - Project1\FYS-STK4155-Prj1_report


### Global variables 

In [2]:
INPUT_DATA = "data/input_data/"  # Path for input data
REPORT_DATA = "data/report_data/" # Path for data ment for the report
REPORT_FIGURES = "figures/" # Path for figures ment for the report

#  Franke function 3D preview
First we plot a 3D plot of the franke function.
The plot is based on the provided code in the assignmentext for plotting the franke function 

In [3]:
# Preview plot of the franke function
%matplotlib
np.random.seed(4155)
y = x = np.arange(0, 1, 0.05)
x, y = np.meshgrid(x,y)
z = FrankeFunction(x, y)
noise_mesh = 0.05 * np.random.randn(z.shape[0], z.shape[1]) # Stochastic noise
z_noisy = z + noise_mesh

fig = plt.figure()
# Ploting frankefunction without noise
ax1 = fig.add_subplot(1,2,1, projection='3d') # Are :)steike
ax1.title.set_text("Plot of the Franke Function")
ax1.view_init(elev=30., azim=-25.0)
ax1.set_xlabel("x"); ax1.set_ylabel("y"); ax1.set_zlabel("z")
surf1 = ax1.plot_surface(x,y,z, cmap=cm.coolwarm, linewidth = 0, antialiased=False)
# Customize the z axis.
ax1.set_zlim(-0.10, 1.40)
ax1.zaxis.set_major_locator(LinearLocator(10))
ax1.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Ploting frankefunction with noise
ax2 = fig.add_subplot(1,2,2, projection='3d')
ax2.title.set_text("Plot of the Franke Function\n(noise added)")
ax2.view_init(elev=30., azim=-25.0)
ax2.set_xlabel("x"); ax2.set_ylabel("y"); ax2.set_zlabel("z")
surf2 = ax2.plot_surface(x,y,z_noisy, cmap=cm.coolwarm, linewidth = 0, antialiased=False)
# Customize the z axis.
ax2.set_zlim(-0.10, 1.40)
ax2.zaxis.set_major_locator(LinearLocator(10))
ax2.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

plt.savefig(f"{REPORT_FIGURES}franke_function_preview.pdf")

Using matplotlib backend: Qt5Agg


# 1 - Ordinary Least Squeares (OLS)

## 1.1 Data
Defining and creating the data


In [4]:
n = 2000 # The number of point in the franke function
x = np.random.rand(n)
y = np.random.rand(n)
z = FrankeFunction(x, y) 
noise = 0.05 * np.random.randn(n) # Stochastic noise
z += noise # Adding stochastic noise

## 1.2 Plot of fit for all degrees before evaluation
 We plot the fit up to degree 6 to get an intuition on the curvature of the fitted models

In [5]:
fig = plt.figure(figsize=(8,8))
degrees = 6
z_train_OLS = pd.DataFrame()
z_hat_train_OLS = pd.DataFrame()
z_test_OLS = pd.DataFrame()
z_hat_test_OLS = pd.DataFrame()

# TODO: Must fix so that training and test data are used. 
# Must evalute model using MSE from traning and test
for degree in range(1, degrees + 1):
    X = create_X(x, y, degree) # Design MatrixS
    model = OLS() # The model
    model.fit(X, z) # Fitting the model
    z_hat = model.predict(X) # predict on train data
    
    # Plot
    ax = fig.add_subplot(3,2, degree, projection='3d')
    ax.view_init(elev=30., azim=-25.0)
    ax.title.set_text(f"OLS/Linear fit of degree{degree}")
    ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")
    ax.scatter3D(y, x, z_hat, c=z_hat ,marker = '.', cmap=cm.coolwarm)
fig.suptitle("OLS fit to the Franke Function")
plt.tight_layout()
plt.savefig(f"{REPORT_FIGURES}franke_function_OLS_fit.pdf")

## 1.3 - Finding degree/model complexity for the optimal OLS fit
Approximate the franke function using ordinary least squares
We estimate the franke functinon using polynomials up to 6th degree. We than look at the MSE scores to look for overfitting. We use the MSE score values from the test data to determine overfit together with the curvature of the evaluation plot

### Confidence intervall 
$$CI_{0.95}(\hat\beta_i) = [\hat\beta_i-1.96 SE(\hat\beta_i), \hat\beta_i+1.96 SE(\hat\beta_i)] =\hat\beta_i \pm 1.96\hat \sigma(\hat\beta_i)$$ 
In order to estimate the variance of the $i$-th beta values: $$\sigma^2 (\beta_i ) = \sigma^2 [(X^{T} X)^{-1}]_{ii}$$
However, $\sigma$ is unkown and can be generaly estimated as followed:
$$\hat\sigma^2 = \frac{\sum_{i=0}^{N-1}(y_i - \hat y_i)^2}{N}$$
For simplification purposes, we N instead of N-p-1 in the denominator.<br>
To get the variance estimate of each $\beta$ component one must calculate the variance with respect to the diagonal elements of $(X^TX)^{-1}$ Estimated standard error is the square root of $\hat\sigma^2$, where the estimate for variance $\hat\sigma^2$ is:
$$\hat\sigma^2 = \frac{\sum_{i=0}^{N-1}(y_i - \hat y_i)^2}{N}(X^TX)^{-1}$$
Where y is the true value, and $\hat y$ being the predicted value. <br>
The variance estimate of each $\hat\beta$ estimate can be written as:
$$\hat\sigma_{\hat\beta_i}^2 = \hat\sigma^2(X^TX)_{i,i}^{-1}$$



In [8]:
degrees = 5
z_train_OLS = pd.DataFrame()
z_hat_train_OLS = pd.DataFrame()
z_test_OLS = pd.DataFrame()
z_hat_test_OLS = pd.DataFrame()

# TODO: Must fix so that training and test data are used. 
# Must evalute model using MSE from traning and test
conf_df = pd.DataFrame()
for degree in range(1, degrees + 1):
    X = create_X(x, y, degree) # Design Matrix
    #X_train, X_test, z_train, z_test = prepare_data(X, z, test_size=0.2, shuffle=True, scale_X=False, scale_t=False)
    X_train, X_test, z_train, z_test = prepare_data(X, z, test_size=0.2, shuffle=True, scale_X=False, scale_t=False, random_state=4155)
    model = OLS() # The model
    model.fit(X_train, z_train) # Fitting the model
    
    z_hat_train = model.predict(X_train) # predict on train data
    z_hat_test = model.predict(X_test) # predict on test data

    # Evaluatation metrics
    # TODO:

    N, P = X.shape
    variance = np.sum(((z_train - z_hat_train) ** 2)) / N
    X_T_X_diag = np.diag(np.linalg.pinv(X.T @ X))
    SE_betas = np.sqrt(variance * X_T_X_diag) 
    betas = model.get_all_betas()
    lower_all_betas = betas - (1.966 * SE_betas)
    upper_all_betas = betas + (1.966 * SE_betas)

    #df = pd.DataFrame(columns=["lower_conf","upper_conf", "beta coefficient" ])

    df = pd.DataFrame.from_dict({"lower_conf":lower_all_betas, "upper_conf":upper_all_betas, 
                                 "beta": betas,  "degree" :degree*betas.shape}, orient='index').T
    conf_df = pd.concat([conf_df,df], axis=0)

    #"beta_no": f"beta{np.arange(betas.shape[0]+1)}",


    """
    conf_intervall_betas = np.zeros((betas.shape[0],2))
    for i in range(betas):
        conf_intervall_betas[i, 0] = beta - 1.966 * SE_betas[i]


    #variance = np.sum((z_train - np.mean(z_hat_train)) ** 2) / n
    #sigma = np.sqrt(variance)
    #sigma = np.sqrt(1/(X.shape[0] - X.shape[1] - 1) * np.sum((z_test - z_hat_test)**2))
    
    
    #variance_estimate = sigma * X_T_X_diag
    #variance_estimate = sigma * np.sqrt(X_T_X_diag) ## sqrt of X_T_X_diag???

    for i, beta in enumerate(betas):
        
        #var_beta = np.diag(sigma *np.linalg.inv((X.T.dot(X)))) # Variance of the beta coefficients
        #variance_estimate_beta_i = sigma * np.sqrt(np.linalg.pinv(X.T @ X))[i,i] # Variance of the beta coefficients
        
        
        SE_beta_i = np.sqrt(variance_estimate_beta_i / N) # Standard error of estimated beta
        lower = beta - 1.966 * SE_beta_i
        upper = beta + 1.966 * SE_beta_i
        df = pd.DataFrame.from_dict({"lower_conf":lower, "upper_conf":upper, "beta": beta, "beta_no": f"beta{i+1}", "degree" :degree}, orient='index').T
        conf_df = pd.concat([conf_df,df], axis=0)

        
        #error = np.mean( np.mean((y_test - y_pred)**2, axis=1, keepdims=True) )
        #bias = np.mean( (y_test - np.mean(y_pred, axis=1, keepdims=True))**2 )
        #variance = np.mean( np.var(y_pred, axis=1, keepdims=True) )
    """

    # Filling up dataframes
    z_train_OLS[degree] = z_train.flatten() 
    z_hat_train_OLS[degree] = z_hat_train.flatten()
    z_test_OLS[degree] = z_test.flatten()
    z_hat_test_OLS[degree] = z_hat_test.flatten()


# MSE calculations for all lambda values
mse_scores_train = ((z_train_OLS - z_hat_train_OLS) ** 2).mean()
mse_scores_test = ((z_test_OLS - z_hat_test_OLS) ** 2).mean()
# R2 calculations for all lambda values
R2_scores_train = 1 - ((z_train_OLS - z_hat_train_OLS) ** 2).sum() / ((z_train_OLS - z_train_OLS.mean())**2).sum() 
R2_scores_test = 1 - ((z_test_OLS - z_hat_test_OLS) ** 2).sum() / ((z_test_OLS - z_test_OLS.mean())**2).sum()  

# Plots
plt.figure(figsize=(12,8))
plt.plot(np.arange(1,degrees+1), mse_scores_train, c="c", label="Training data")
plt.plot(np.arange(1,degrees+1), mse_scores_test, c="m", label="Test data")
plt.xlabel("Model complexity - Number of polynomial degrees")
plt.ylabel("MSE")
plt.title("Training evaluation on OLS regression fit")
plt.legend()
plt.tight_layout()
plt.savefig(f"{REPORT_FIGURES}franke_function_OLS_evaluate_fit.pdf")

## 1.4 Analysis of plots and training metrics
Do the analysis.......

Conclusion:<br>
Based on the analysis, we conclude that a model complexity of degree 5 yields the most optimal fit.

## 1.5 plot of the model using the most optimal parameters

In [ ]:
degree_optimal = 5
X = create_X(x, y, degree_optimal) # Design Matrix
model = OLS() # The model
model.fit(X, z) # Fitting the model
z_hat = model.predict(X) # predict on train data

# Plot
fig = plt.figure()#figsize=(8,8))
ax = plt.axes(projection='3d')
ax.title.set_text(f"OLS regression fit to the Franke Function\noptimal degree {degree_optimal},")
#ax.view_init(elev=5., azim=85.0)
ax.view_init(elev=30., azim=-25.0)
ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")
ax.scatter3D(y, x, z_hat, c=z_hat ,marker = '.', cmap=cm.coolwarm)
plt.savefig(f"{REPORT_FIGURES}franke_function_OLS_best_fit.pdf")